# A refinement of 1st version of pledge information extraction

In [ ]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
import multiprocessing
import pickle  
import spacy
import re
import string
import deepdish as dd

In [ ]:
nlp = spacy.load('en')

In [ ]:
info_dict = dd.io.load('cleaned_info.h5')

In [ ]:
info = pd.DataFrame.from_dict(info_dict,orient='index')

In [ ]:
def parallel(function, loop):
    core = multiprocessing.cpu_count()
    Parallel(n_jobs= core, backend='threading')(delayed(function)(i) for i in loop)

In [ ]:
plginfo = info[1]
link = info_dict[2][0]

In [ ]:
result = {}

In [185]:
def extract(ind):
    text = plginfo[ind]
    FName = link[ind]
    token = nlp(text)
    sents = [str(w) for w in token.sents]
    sents_plg = [x for x in sents if ('share' or 'Share' or 'SHARE') in x and  ('pledg'or 'Pledg'or'PLEDG') in x]
    tags = [(i, x, x.tag_) for i, x in enumerate(token)]
    number = [str(s[1]) for s in tags if s[2] == 'CD' and ',' in str(s[1])]
    length = len(sents_plg)
    if number != []:
        entities = []
        share_num = []
        pair = []
        for sent in sents_plg:
            tk = nlp(sent)
            ents = list(tk.ents)
            e = [str(p) for p in ents if p.label_ == 'PERSON']
            s = [str(r) for r in ents if (r.label_ == 'CARDINAL' and str(r) in number)]
            entities = list(e)
            share_num = list(s)
            match_share_name = list(zip(e,s))
            pair.append(match_share_name)
    else:
        entities = 'None'
        share_num = 'None'
        pair = 'None'
    result[FName] = (FName,length,entities,number,share_num,pair)
    return 

In [172]:
def process(ind):
    try:
        extract(ind)
        print(ind)
    except:
        print(ind, 'error')
        result[ind] = ()

In [ ]:
parallel(process, plginfo.index)

In [ ]:
dd.io.save('refined_pldinfo.h5', result, compression = 'blosc')